In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Define equities and parameters
equities = ["AAPL", "GOOG", "NVDA", "META", "TSLA"]
expiration_date = "2025-02-23"  # Example expiration date
strike_offset = 0.1  # 10% OTM for both calls and puts
premium = 5  # Example premium for each option

portfolio = []

# Loop through equities to create strangles
for equity in equities:
    stock = yf.Ticker(equity)
    current_price = stock.history(period="1d")["Close"].iloc[-1]

    # Calculate strike prices
    call_strike = round(current_price * (1 + strike_offset), 2)
    put_strike = round(current_price * (1 - strike_offset), 2)

    # Create the strangle structure
    strangle = {
        "Equity": equity,
        "Current Price": round(current_price, 2),
        "Call Strike": call_strike,
        "Put Strike": put_strike,
        "Expiration Date": expiration_date
    }
    portfolio.append(strangle)

# Convert portfolio to a DataFrame
portfolio_df = pd.DataFrame(portfolio)
print(portfolio_df)

# Visualize PnL for all strangles on one plot
plt.figure(figsize=(10, 6))

for index, row in portfolio_df.iterrows():
    equity = row['Equity']
    current_price = row['Current Price']
    call_strike = row['Call Strike']
    put_strike = row['Put Strike']

    # Define price range for PnL calculation
    price_range = np.linspace(current_price * 0.5, current_price * 1.5, 100)

    # Calculate PnL for call and put options
    call_pnl = np.maximum(price_range - call_strike, 0) - premium
    put_pnl = np.maximum(put_strike - price_range, 0) - premium
    total_pnl = call_pnl + put_pnl

    # Plot PnL
    plt.plot(price_range, total_pnl, label=f"{equity} Total PnL")

plt.axhline(0, color='black', linestyle='--', linewidth=0.8)
plt.title("Strangle PnL for All Equities")
plt.xlabel("Stock Price at Expiration")
plt.ylabel("PnL")
plt.legend()
plt.grid()
plt.show()
